# Evaluate models on IPN cohorts

In [194]:
import os
import pandas as pd
import numpy as np
import math
import scipy.stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import statsmodels.formula.api as smf
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

In [171]:
cohort_path = "/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/nlst/nlst_ipn_v2.csv"
cohort_df = pd.read_csv(cohort_path)
cohort_df

,pid,id,session,age,sex,race,education,bmi,phist,fhist,...,smo_intensity,smo_duration,spiculation,upper_lobe,nodule_size,nodule_type,nodule_count,with_image,with_marker,lung_cancer
0,100004,100004time2000,1,60,1,1,4,29.414135,False,False,...,40,23.0,0,1,4.0,2.0,1,True,True,0.0
1,100012,100012time2000,1,61,0,1,6,22.240116,False,False,...,20,39.0,1,1,15.0,0.0,1,True,True,1.0
2,100019,100019time2000,1,61,1,1,4,23.962608,False,False,...,40,43.0,1,1,14.0,2.0,1,True,True,0.0
3,100026,100026time2001,2,57,1,1,3,35.146505,False,False,...,30,41.0,0,0,5.0,2.0,2,True,True,0.0
4,100035,100035time2001,2,55,0,1,3,22.096473,False,False,...,20,38.0,0,0,5.0,2.0,1,True,True,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5784,218391,218391time2000,0,66,1,1,6,23.674792,False,False,...,20,40.0,0,1,17.0,1.0,4,True,True,1.0
5785,218499,218499time1999,0,63,1,1,2,23.110395,False,False,...,30,48.0,1,1,12.0,1.0,1,True,True,1.0
5786,218510,218510time2001,2,64,1,1,6,27.802713,False,False,...,30,47.0,0,1,14.0,2.0,4,True,True,1.0
5787,218705,218705time2001,2,68,1,1,4,37.305733,False,False,...,40,43.0,0,1,6.0,2.0,2,True,True,0.0


## DL models

In [172]:
dls_path = "/home/local/VANDERBILT/litz/data/nlst/DeepLungScreening/pred/ipn_pred_v2.csv"
dls = pd.read_csv(dls_path)
dls['dls_risk'] = dls['pred']
cohort_df = cohort_df.merge(dls[['id', 'dls_risk']], on='id')


In [173]:
cscnn_path = "/home/local/VANDERBILT/litz/github/MASILab/DSB2017/models/config_nlst_ipn_1211/preds.csv"
cscnn = pd.read_csv(cscnn_path)
cscnn['cscnn_risk'] = cscnn['pred']
cscnn['id'] = cscnn['id'].apply(lambda x: x.split("'")[1])
cohort_df = cohort_df.merge(cscnn[['id', 'cscnn_risk']], on='id')

In [175]:
dlstm_path = "/home/local/VANDERBILT/litz/github/MASILab/RNNLung/compare/tumor_NLST/models/1130_nlst_ipn/pred.csv"
dlstm = pd.read_csv(dlstm_path)
dlstm['dlstm_risk'] = dlstm['pred']
cohort_df = cohort_df.merge(dlstm[['pid', 'dlstm_risk']], on='pid', how='left')

In [176]:
tdvit_path = "/home/local/VANDERBILT/litz/github/MASILab/time-distance-transformer/models/1109_nlst_ipn_td/pred.csv"
tdvit = pd.read_csv(tdvit_path)
tdvit['tdvit_risk'] = tdvit['pred']
cohort_df = cohort_df.merge(tdvit[['PID', 'tdvit_risk']], left_on='pid', right_on='PID', how='left')

## Brock

In [178]:
def Brock(age, sex, fhist, emphysema, nodule_size, upper_lobe, nodule_count, spiculation, nodule_nonsolid, nodule_partsolid, nodule_solid):
    """Full Model, with Spiculation from DOI: 10.1056/NEJMoa1214726"""
    age_var = 0.0287*(age - 62)
    sex_var = 0.6011* sex
    fhist_var = 0.2961*fhist
    emphysema_var = 0.2953*emphysema
    nodule_size_var = -5.3854* (np.sqrt(10/max(nodule_size -4, 1e-6))) -1.58113883 # nonlinear transform
    nodule_nonsolid_var = -0.1276*nodule_nonsolid
    nodule_partsolid_var = 0.3770*nodule_partsolid
    nodule_solid_var = 0*nodule_solid
    upper_lobe_var = 0.6581*upper_lobe
    nodule_count_var = -0.0824*(nodule_count - 4)
    spiculation_var = 0.7729*spiculation
    res = age_var + sex_var + fhist_var + emphysema_var + nodule_size_var + nodule_nonsolid_var + nodule_partsolid_var \
        + nodule_solid_var + upper_lobe_var + nodule_count_var + spiculation_var - 6.7892
    res = np.exp(res) / (1 + np.exp(res))
    return res


In [179]:
# brock_path = "/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/nlst_brock_v1.csv"
# brock_df = pd.read_csv(brock_path, dtype={'pid':str})
# brock_df = cohort_df

features = ['age', 'sex', 'fhist', 'emphysema', 
'spiculation', 'upper_lobe', 'nodule_size', 'nodule_type', 'nodule_count']
cohort_df['lung_cancer'] = cohort_df['lung_cancer'].astype(int)
features_df = cohort_df[features]

In [180]:

x = pd.get_dummies(features_df, columns=['nodule_type'])
enc_features = x.columns
# Multiple linear imputation
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(x)
imp_x = pd.DataFrame(imp.transform(x), columns=enc_features)

brock_enc = pd.merge(cohort_df[['pid','id','session','lung_cancer']], imp_x, left_index=True, right_index=True)

# nodule size must be >=4 to be considered nodule
brock_enc = brock_enc[brock_enc['nodule_size']>=4]

In [181]:
brock_enc['brock_risk'] = brock_enc.apply(lambda x: Brock(age=x['age'], sex=x['sex'], fhist=x['fhist'], emphysema=x['emphysema'], \
    nodule_size=x['nodule_size'], upper_lobe=x['upper_lobe'], nodule_count=x['nodule_count'], spiculation=x['spiculation'], \
    nodule_nonsolid=x['nodule_type_0.0'], nodule_partsolid=x['nodule_type_1.0'], nodule_solid=x['nodule_type_2.0']), axis=1)

y, y_hat = brock_enc['lung_cancer'].to_numpy().astype(float), brock_enc['brock_risk'].to_numpy().astype(float)
roc = roc_auc_score(y, y_hat)
print(f"AUC: {roc}")

cohort_df = cohort_df.merge(brock_enc[['pid', 'brock_risk']], on='pid')

AUC: 0.8411565736761094


In [182]:
# fitting a lienar model instead of using original weights

# scalars = ['age', 'nodule_size', 'nodule_count']
# brock_enc[scalars] = brock_enc[scalars].astype(float)
# brock_enc[scalars].min()
# brock_enc[scalars] = (brock_enc[scalars] - brock_enc[scalars].min())/(brock_enc[scalars].max() - brock_enc[scalars].min())

# import statsmodels.formula.api as smf
# # formula_str = 'age+bmi+copd+phist+fhist+smo_status+quit_time+pkyr+C(race, Treatment(reference=1))+C(education, Treatment(reference=1))'
# formula_str = 'age+sex+fhist+emphysema+spiculation+upper_lobe+nodule_size+nodule_type+nodule_count'
# smf_lr = smf.logit(f"lung_cancer ~ {formula_str}", data=brock_enc).fit()
# smf_lr.summary()

# odds = pd.DataFrame({"OR": smf_lr.params, "Lower CI": smf_lr.conf_int()[0], "Upper CI": smf_lr.conf_int()[1]})
# odds = np.exp(odds)
# odds["OR"] = odds["OR"].apply(lambda x: '{:.4f}'.format(x))

# # report model performance
# X, y = brock_enc[features], brock_enc['lung_cancer'].to_numpy().ravel()
# y_prob = smf_lr.predict(brock_enc[features])
# y_hat = round(y_prob)

# # accuracy_score
# roc = roc_auc_score(y, y_prob)
# report = classification_report(y, y_hat)
# acc = accuracy_score(y, y_hat)
# print(f"AUC: {roc}")
# print(report)
# print(f"Accuracy: {acc}")
# print(f"Chi-squared statistic: {smf_lr.llr} p:{smf_lr.llr_pvalue}")

## Mayo

In [183]:
def Mayo(age, phist, smo_status, nodule_size, spiculation, upper_lobe):
    """Full Model, with Spiculation from DOI: 10.1056/NEJMoa1214726"""
    age_var = 0.0391*age
    phist_var = 1.3388*int(phist)
    smo_status_var = 0.7917*smo_status
    nodule_size_var = 0.1274*nodule_size # nonlinear transform
    upper_lobe_var = 0.7838*upper_lobe
    spiculation_var = 1.0407*spiculation
    res = age_var + phist_var+ smo_status_var + nodule_size_var + upper_lobe_var + spiculation_var - 6.8272
    res = np.exp(res) / (1 + np.exp(res))
    return res

In [184]:
# mayo_path = "/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/nlst_mayo_v1.csv"
# mayo_df = pd.read_csv(mayo_path, dtype={'pid':str})

features = ['age', 'phist', 'smo_status', 'spiculation', 'upper_lobe', 'nodule_size', 'lung_cancer']
mayo_df = cohort_df.copy()
mayo_df['lung_cancer'] = mayo_df['lung_cancer'].astype(int)
features_df = mayo_df[features]
x = features_df
enc_features = x.columns
# Multiple linear imputation
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(x)
imp_x = pd.DataFrame(imp.transform(x), columns=enc_features)

mayo_enc = pd.merge(mayo_df.drop(columns=['nodule_size']), imp_x['nodule_size'], left_index=True, right_index=True)


In [185]:
mayo_enc['mayo_risk'] = mayo_enc.apply(lambda x: Mayo(age=x['age'], phist=x['phist'], smo_status=x['smo_status'],
    nodule_size=x['nodule_size'], spiculation=x['spiculation'], upper_lobe=x['upper_lobe']), axis=1)

y, y_hat = mayo_enc['lung_cancer'].to_numpy().astype(float), mayo_enc['mayo_risk'].to_numpy().astype(float)
roc = roc_auc_score(y, y_hat)
print(f"AUC: {roc}")

cohort_df = cohort_df.merge(mayo_enc[['pid', 'mayo_risk']], on='pid')

AUC: 0.8328448694205955


In [186]:
# fitting a linear model instead of using original weights

# scalars = ['age', 'nodule_size']
# mayo_enc[scalars] = mayo_enc[scalars].astype(float)
# mayo_enc[scalars].min()
# mayo_enc[scalars] = (mayo_enc[scalars] - mayo_enc[scalars].min())/(mayo_enc[scalars].max() - mayo_enc[scalars].min())

# import statsmodels.formula.api as smf
# # formula_str = 'age+bmi+copd+phist+fhist+smo_status+quit_time+pkyr+C(race, Treatment(reference=1))+C(education, Treatment(reference=1))'
# formula_str = 'age+phist+spiculation+upper_lobe+nodule_size'
# smf_lr = smf.logit(f"lung_cancer ~ {formula_str}", data=mayo_enc).fit()
# smf_lr.summary()

# odds = pd.DataFrame({"OR": smf_lr.params, "Lower CI": smf_lr.conf_int()[0], "Upper CI": smf_lr.conf_int()[1]})
# odds = np.exp(odds)
# odds["OR"] = odds["OR"].apply(lambda x: '{:.4f}'.format(x))
# odds

In [187]:
# # report model performance
# X, y = mayo_enc[features], mayo_enc['lung_cancer'].to_numpy().ravel()
# y_prob = smf_lr.predict(mayo_enc[features])
# y_hat = round(y_prob)

# # accuracy_score
# roc = roc_auc_score(y, y_prob)
# report = classification_report(y, y_hat)
# acc = accuracy_score(y, y_hat)
# print(f"AUC: {roc}")
# print(report)
# print(f"Accuracy: {acc}")
# print(f"Chi-squared statistic: {smf_lr.llr} p:{smf_lr.llr_pvalue}")

### PLCO2012

In [188]:
def PLCOm2012(age, race, education, body_mass_index, copd, phist, fhist,
              smoking_status, smoking_intensity, duration, quit_time):
    def get_race(x):
        d = {
            0: 0, # unknown
            1: 0, # white
            2: 0.3944778, # black
            3: -0.7434744, # hispanic
            4: -0.466585, # asian
            5: 0, # american indian or alaskan native
            6: 1.027152 # Native hawaiian or pacific islander
        }
        return d[x]
    age_item = 0.0778868 * (age - 62)
    edu_item = -0.0812744 * (education - 4)
    bmi_item = -0.0274194 * (body_mass_index - 27)
    copd_item = 0.3553063 * int(copd)
    phist_item = 0.4589971 * int(phist)
    fhist_item = 0.587185 * int(fhist)
    sstatus_item = 0.2597431 * (smoking_status - 1)
    sint_item = - 1.822606 * (10 / smoking_intensity - 0.4021541613)
    duration_item = 0.0317321 * (duration - 27)
    qt_item = -0.0308572 * (quit_time - 10)
    res = age_item + get_race(race) + edu_item + bmi_item \
          + copd_item + phist_item + fhist_item + sstatus_item \
          + sint_item + duration_item + qt_item - 4.532506
    res = np.exp(res) / (1 + np.exp(res))
    return res

In [189]:
plco = cohort_df.copy()

features = ['age', 'race', 'education',  'bmi',  'copd', 'phist', 'fhist', 'smo_status', 'smo_intensity', 'smo_duration', 'quit_time', 'pkyr']
label = ['lung_cancer']
features_df = plco[features]

x = features_df
enc_features = x.columns
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(x)
imp_x = pd.DataFrame(imp.transform(x), columns=enc_features)
plco_enc = pd.merge(plco.drop(columns=['bmi', 'copd']), imp_x[['copd','bmi']], left_index=True, right_index=True)

In [190]:
plco_enc['plco_risk'] = plco_enc.apply(lambda item: PLCOm2012(age = item['age'], race = item['race'], education = item['education'], body_mass_index = item['bmi'], \
    copd = item['copd'], phist = item['phist'], fhist = item['fhist'], smoking_status = item['smo_status'], smoking_intensity = item['smo_intensity'], \
    duration = item['smo_duration'], quit_time = item['quit_time']), axis=1)

y, y_hat = plco_enc['lung_cancer'].to_numpy().astype(float), plco_enc['plco_risk'].to_numpy().astype(float)
roc = roc_auc_score(y, y_hat)
print(f"AUC: {roc}")

cohort_df = cohort_df.merge(plco_enc[['pid', 'plco_risk']], on='pid')

AUC: 0.6632393119926432


### Save evaluations

In [192]:
pred_path = "/home/local/VANDERBILT/litz/data/nlst/DeepLungScreening/pred/ipn_eval.csv"
cohort_df.to_csv(pred_path, index=False)

### Eval cross sectional models

In [193]:
cohort_df

,pid,id,session,age,sex,race,education,bmi,phist,fhist,...,with_marker,lung_cancer,dls_risk,cscnn_risk,dlstm_risk,PID,tdvit_risk,brock_risk,mayo_risk,plco_risk
0,100004,100004time2000,1,60,1,1,4,29.414135,False,False,...,True,0,0.011661,0.493369,0.229992,100004.0,0.224410,0.000000e+00,0.039627,0.006573
1,100012,100012time2000,1,61,0,1,6,22.240116,False,False,...,True,1,0.198917,0.934444,0.883780,100012.0,0.789656,6.248166e-06,0.521262,0.015806
2,100019,100019time2000,1,61,1,1,4,23.962608,False,False,...,True,0,0.040926,0.488559,0.248859,100019.0,0.481735,1.354013e-05,0.302800,0.022187
3,100026,100026time2001,2,57,1,1,3,35.146505,False,False,...,True,0,0.022328,0.485828,0.187423,100026.0,0.317768,1.733889e-11,0.018679,0.011595
4,100035,100035time2001,2,55,0,1,3,22.096473,False,False,...,True,0,0.011342,0.242707,0.115808,100035.0,0.229410,1.309397e-11,0.037399,0.012338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5780,218391,218391time2000,0,66,1,1,6,23.674792,False,False,...,True,1,0.247947,0.999059,NaN,NaN,NaN,1.592910e-05,0.214670,0.013153
5781,218499,218499time1999,0,63,1,1,2,23.110395,False,False,...,True,1,0.071401,0.445031,0.224025,218499.0,0.232901,8.240848e-06,0.445492,0.043729
5782,218510,218510time2001,2,64,1,1,6,27.802713,False,False,...,True,1,0.253550,0.999598,0.803832,218510.0,0.778432,5.321027e-06,0.147116,0.032387
5783,218705,218705time2001,2,68,1,1,4,37.305733,False,False,...,True,0,0.030812,0.538547,0.223053,218705.0,0.219423,9.045854e-09,0.138417,0.037178


In [204]:
random_seed=56
n_bootstrap = 1000 # number of bootstrap samples with replacement (the samples are same size as cohort)

model_names = ['mayo', 'brock', 'plco', 'cscnn', 'dls']

def compute_model_auc(sample, model):
    y = sample['lung_cancer']
    y_prob = sample[f"{model}_risk"]
    return roc_auc_score(y, y_prob)

def compute_ci(data, confidence=0.95):
    a = 1.0*np.array(data)
    n = len(a)
    mu, se = np.mean(a), scipy.stats.sem(a)
    h = se*scipy.stats.t.ppf((1+confidence)/2.0, n-1)
    # print(mu)
    return mu, mu-h, mu+h

nonnull_cohort = cohort_df[~cohort_df['plco_risk'].isnull() & ~cohort_df['mayo_risk'].isnull() 
    & ~cohort_df['brock_risk'].isnull() & ~cohort_df['cscnn_risk'].isnull() 
    & ~cohort_df['dls_risk'].isnull()]

dfrows= []
for model in model_names:
    # calculate 95% CI with bootstrap sampling
    aucs = []
    for i in range(n_bootstrap):
        sample = nonnull_cohort.sample(frac=1.0, replace=True)
        aucs.append(compute_model_auc(sample, model))

    mean_auc, ci_low, ci_high = compute_ci(aucs)
    dfrows.append({'model': model, 'mean_AUC': mean_auc, 'ci_low':ci_low, 'ci_high':ci_high, 'std': np.std(aucs)})

metrics = pd.DataFrame(dfrows)
metrics

,model,mean_AUC,ci_low,ci_high,std
0,mayo,0.832448,0.831941,0.832956,0.008175
1,brock,0.840764,0.840247,0.841282,0.008332
2,plco,0.664230,0.663554,0.664906,0.010882
3,cscnn,0.836016,0.835468,0.836565,0.008839
4,dls,0.881397,0.880911,0.881883,0.007832


### Eval longitudinal models

In [205]:
nonnull_cohort = cohort_df[~cohort_df['tdvit_risk'].isnull() & ~cohort_df['dlstm_risk'].isnull()]

model_names = ['tdvit', 'dlstm']

dfrows= []
for model in model_names:
    # calculate 95% CI with bootstrap sampling
    aucs = []
    for i in range(n_bootstrap):
        sample = nonnull_cohort.sample(frac=1.0, replace=True)
        aucs.append(compute_model_auc(sample, model))

    mean_auc, ci_low, ci_high = compute_ci(aucs)
    dfrows.append({'model': model, 'mean_AUC': mean_auc, 'ci_low':ci_low, 'ci_high':ci_high, 'std':np.std(aucs)})

metrics = pd.DataFrame(dfrows)
metrics

,model,mean_AUC,ci_low,ci_high,std
0,tdvit,0.843485,0.842767,0.844203,0.011562
1,dlstm,0.890725,0.890133,0.891317,0.009538
